In [11]:
%%time

import os
import boto3
import pandas as pd
import numpy as np

import sagemaker
from sagemaker import get_execution_role



CPU times: user 98.6 ms, sys: 11.2 ms, total: 110 ms
Wall time: 300 ms


In [7]:
region = boto3.Session().region_name
role = get_execution_role()

In [8]:
try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [9]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [ ]:
s3://mysagemakertutorial/sagemaker/DEMO-xgboost-dm/output/xgboost-2021-12-19-11-18-08-116/output/model.tar.gz

In [ ]:
https://mysagemakertutorial.s3.us-west-2.amazonaws.com/sagemaker/DEMO-xgboost-dm/output/xgboost-2021-12-19-11-18-08-116/output/model.tar.gz
    

In [13]:
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [14]:
%%time
from time import gmtime, strftime

# model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = "xgboost-2021-12-19-11-18-08-116"

# model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)
model_url = "https://mysagemakertutorial.s3.us-west-2.amazonaws.com/sagemaker/DEMO-xgboost-dm/output/xgboost-2021-12-19-11-18-08-116/output/model.tar.gz"
sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

https://mysagemakertutorial.s3.us-west-2.amazonaws.com/sagemaker/DEMO-xgboost-dm/output/xgboost-2021-12-19-11-18-08-116/output/model.tar.gz
arn:aws:sagemaker:us-west-2:191141918684:model/xgboost-2021-12-19-11-18-08-116
CPU times: user 67.4 ms, sys: 11.1 ms, total: 78.5 ms
Wall time: 470 ms


In [15]:
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoostEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-XGBoostEndpointConfig-2021-12-20-15-51-14
Endpoint Config Arn: arn:aws:sagemaker:us-west-2:191141918684:endpoint-config/demo-xgboostendpointconfig-2021-12-20-15-51-14


In [16]:
%%time
import time

endpoint_name = "DEMO-XGBoostEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

DEMO-XGBoostEndpoint-2021-12-20-15-51-37
arn:aws:sagemaker:us-west-2:191141918684:endpoint/demo-xgboostendpoint-2021-12-20-15-51-37
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:191141918684:endpoint/demo-xgboostendpoint-2021-12-20-15-51-37
Status: InService
CPU times: user 73.4 ms, sys: 4.23 ms, total: 77.6 ms
Wall time: 4min


In [1]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

NameError: name 'xgb' is not defined

In [10]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

NameError: name 'xgb_predictor' is not defined

In [12]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

NameError: name 'predictions_array' is not defined

In [22]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

sess = sagemaker.Session()

role = get_execution_role()

In [23]:
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")



The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [28]:
from sagemaker.serializers import CSVSerializer

lang_id = sagemaker.Model(
    image_uri=container, model_data=model_url, role=role, sagemaker_session=sess
)
lang_id.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

predictor = sagemaker.Predictor(
    endpoint_name=lang_id.endpoint_name,
    sagemaker_session=sess,
    serializer=CSVSerializer()
)

-----!

In [25]:

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array

payload = {"instances": test_data_array}
predictions = predictor.predict(test_data_array)
print(predictions)


In [29]:
payload = {"instances": test_data_array}
predictions = predictor.predict(test_data_array)
print(predictions)

b'0.04957985132932663,0.05882718414068222,0.04969020560383797,0.34993410110473633,0.10693199187517166,0.07036889344453812,0.026261720806360245,0.03152993693947792,0.030292263254523277,0.03133094310760498,0.10844358801841736,0.04390738904476166,0.03960980847477913,0.05338261276483536,0.0581413172185421,0.02799060009419918,0.07738287001848221,0.04026491940021515,0.08274780213832855,0.09295079112052917,0.045896176248788834,0.04450158402323723,0.028775816783308983,0.08481450378894806,0.06951882690191269,0.05732080340385437,0.0522407591342926,0.03351811319589615,0.04485239461064339,0.03136540204286575,0.12844596803188324,0.008699812926352024,0.09145842492580414,0.03147318959236145,0.04232644662261009,0.06437798589468002,0.2519811689853668,0.04121432825922966,0.5551555752754211,0.08091937005519867,0.05408147722482681,0.04954887926578522,0.031001782044768333,0.07761570811271667,0.039496537297964096,0.06743523478507996,0.10268549621105194,0.06425436586141586,0.12919919192790985,0.0825175121426